In [3]:
import pickle
import os
import requests
from pathlib import Path
import nltk
from nltk import sent_tokenize, word_tokenize, regexp_tokenize
from nltk.corpus import stopwords
import pymorphy2
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [4]:
path_df_fin = "pickles/raw_dataset_fin.pickle"
path_df_inv = "pickles/raw_dataset_inv.pickle"

with open(path_df_fin, 'rb') as data:
    df_fin = pickle.load(data)
    
with open(path_df_inv, 'rb') as data:
    df_inv = pickle.load(data)

In [5]:
df_fin.head()

,id,url,dateRFC,date,title,intro,text,fintech
0,10002,https://vc.ru/flood/10002-inter-yo,"Mon, 17 Aug 2015 20:56:33 +0300",1439834193,Банк «Интеркоммерц» выпустил мобильное приложе...,"Банк «Интеркоммерц», на основе которого работа...","Банк «Интеркоммерц», на основе которого работа...",1
1,10293,https://vc.ru/flood/10293-paypal-me,"Tue, 01 Sep 2015 18:52:52 +0300",1441122772,PayPal запустила в России сервис личных страни...,Платёжная система PayPal запустила в России и ...,Платёжная система PayPal запустила в России и ...,1
2,10768,https://vc.ru/flood/10768-kassa-tinkoff,"Thu, 24 Sep 2015 12:17:25 +0300",1443086245,«Яндекс.Касса» позволит покупать товары в кред...,В сервисе «Яндекс.Касса» появилась возможность...,В сервисе «Яндекс.Касса» появилась возможность...,1
3,10987,https://vc.ru/flood/10987-paybylink-yamoney,"Mon, 05 Oct 2015 16:07:07 +0300",1444050427,«Яндекс.Деньги» предложили интернет-магазинам ...,Компания «Яндекс.Деньги» запустила тестировани...,Компания «Яндекс.Деньги» запустила тестировани...,1
4,11002,https://vc.ru/flood/11002-linkedin-13mln,"Tue, 06 Oct 2015 11:14:53 +0300",1444119293,LinkedIn выплатит $13 млн за рассылку приглаше...,Соцсеть LinkedIn согласилась выплатить $13 млн...,Соцсеть LinkedIn согласилась выплатить $13 млн...,1


In [6]:
df_fin.loc[0]['text']

'Банк «Интеркоммерц», на основе которого работает сервис «Рокетбанк», выпустил мобильное приложение для управления счетами «ё-банк». Об этом сообщили представители компании. Приложение «ё-банк» позволяет узнать расположение ближайших офисов, проверить депозиты и счета, а также оплатить покупки и услуги, перевести средства на счёт другого человека, погасить кредит и провести другие операции. «ё-банк» выпущен в версиях для iOS и Android.     «Сегодня мы с вами живем в 21 веке, времени когда людям важно иметь возможность управлять своими финансами дистанционно, не посещая отделение банка», — сообщил начальник управления организации проектов банка «Интеркоммерц» Юрий Соловкин. Он добавил, что в будущем в приложении появятся новые функции.  На базе основанного в 1991 году «Интеркоммерца» работает банковский сервис «Рокетбанк». Он предоставляет приложение для управления счетами, а счета клиентов открыты в «Интеркоммерце». \\#новость #мобильное\\_приложение #рокетбанк #интеркоммерц #ё\\_банк 

In [7]:
url_stopwords_ru = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ru/master/stopwords-ru.txt"


def get_text(url, encoding='utf-8', to_lower=True):
    url = str(url)
    if url.startswith('http'):
        r = requests.get(url)
        if not r.ok:
            r.raise_for_status()
        return r.text.lower() if to_lower else r.text
    elif os.path.exists(url):
        with open(url, encoding=encoding) as f:
            return f.read().lower() if to_lower else f.read()
    else:
        raise Exception('parameter [url] can be either URL or a filename')


def normalize_tokens(tokens):
    morph = pymorphy2.MorphAnalyzer()
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def remove_stopwords(tokens, stopwords=None, min_length=4):
    if not stopwords:
        return tokens
    stopwords = set(stopwords)
    tokens = [tok
              for tok in tokens
              if tok not in stopwords and len(tok) >= min_length]
    return tokens


def tokenize_n_lemmatize(
    text, stopwords=None, normalize=True, 
    regexp=r'(?u)\b\w{4,}\b'):
    words = [w for sent in sent_tokenize(text)
             for w in regexp_tokenize(sent, regexp)]
    if normalize:
        words = normalize_tokens(words)
    if stopwords:
        words = remove_stopwords(words, stopwords)
    return words

stopwords_ru = get_text(url_stopwords_ru).splitlines()

#text = df_fin.loc[0]['text']
#words = tokenize_n_lemmatize(text, stopwords=stopwords_ru)


In [8]:
# add progress bar
def proc_df(df):
    nrows = len(df)
    text_parsed_list = []
    for row in range(0, nrows):
        text = df.loc[row]['title'] + ' ' + df.loc[row]['text']
        text = text.replace('_', ' ')
        text_parsed_list.append(' '.join(tokenize_n_lemmatize(text)))
    df['text_parsed'] = text_parsed_list

In [9]:
proc_df(df_fin)

In [10]:
proc_df(df_inv)

In [11]:
df_fin.head()

,id,url,dateRFC,date,title,intro,text,fintech,text_parsed
0,10002,https://vc.ru/flood/10002-inter-yo,"Mon, 17 Aug 2015 20:56:33 +0300",1439834193,Банк «Интеркоммерц» выпустил мобильное приложе...,"Банк «Интеркоммерц», на основе которого работа...","Банк «Интеркоммерц», на основе которого работа...",1,банк интеркоммерц выпустить мобильный приложен...
1,10293,https://vc.ru/flood/10293-paypal-me,"Tue, 01 Sep 2015 18:52:52 +0300",1441122772,PayPal запустила в России сервис личных страни...,Платёжная система PayPal запустила в России и ...,Платёжная система PayPal запустила в России и ...,1,paypal запустить россия сервис личный страница...
2,10768,https://vc.ru/flood/10768-kassa-tinkoff,"Thu, 24 Sep 2015 12:17:25 +0300",1443086245,«Яндекс.Касса» позволит покупать товары в кред...,В сервисе «Яндекс.Касса» появилась возможность...,В сервисе «Яндекс.Касса» появилась возможность...,1,яндекс касса позволить покупать товар кредит ч...
3,10987,https://vc.ru/flood/10987-paybylink-yamoney,"Mon, 05 Oct 2015 16:07:07 +0300",1444050427,«Яндекс.Деньги» предложили интернет-магазинам ...,Компания «Яндекс.Деньги» запустила тестировани...,Компания «Яндекс.Деньги» запустила тестировани...,1,яндекс деньга предложить интернет магазин выст...
4,11002,https://vc.ru/flood/11002-linkedin-13mln,"Tue, 06 Oct 2015 11:14:53 +0300",1444119293,LinkedIn выплатит $13 млн за рассылку приглаше...,Соцсеть LinkedIn согласилась выплатить $13 млн...,Соцсеть LinkedIn согласилась выплатить $13 млн...,1,linkedin выплатить рассылка приглашение соцсет...


In [12]:
df_inv.head()

,id,url,dateRFC,date,title,intro,text,investment,text_parsed
0,15512,https://vc.ru/finance/15512-volkov-technologies,"Mon, 16 May 2016 19:29:34 +0300",1463416174,"VR, p2p-кредитование, искусственный интеллект ...",Сооснователь международного интернет-холдинга ...,Сооснователь международного интернет-холдинга ...,1,кредитование искусственный интеллект стоить ин...
1,23445,https://vc.ru/finance/23445-boyko-fintech,"Tue, 25 Apr 2017 16:20:54 +0300",1493126454,Экс-владелец сети казино «Вулкан» Олег Бойко р...,Инвестхолдинг Finstar российского миллиардера ...,Инвестхолдинг Finstar российского миллиардера ...,1,владелец сеть казино вулкан олег бойко решить ...
2,25881,https://vc.ru/tribuna/25881-frii-semantic,"Thu, 17 Aug 2017 15:31:43 +0300",1502973103,Разработчик сервиса для исследования медпрепар...,Фонд развития интернет-инициатив вместе с част...,Фонд развития интернет-инициатив вместе с част...,1,разработчик сервис исследование медпрепарат se...
3,26020,https://vc.ru/tribuna/26020-frii-docsinbox-15m,"Thu, 24 Aug 2017 12:05:14 +0300",1503565514,ФРИИ вложит 15 млн рублей в сервис автоматизац...,Фонд развития интернет-инициатив договорился о...,Фонд развития интернет-инициатив договорился о...,1,фрий вложить рубль сервис автоматизация докуме...
4,26654,https://vc.ru/flood/26654-sleduyushchaya-volna...,"Sun, 24 Sep 2017 16:02:10 +0300",1506258130,Следующая волна в финтехе — не за новыми старт...,Финтех-инвестор Владислав Солодкий — о потенци...,Финтех-инвестор Владислав Солодкий — о потенци...,1,следующий волна финтех новый стартап финтех ин...


In [13]:
X_train_inv, X_test_inv, y_train_inv, y_test_inv = train_test_split(df_inv['text_parsed'], 
                                                    df_inv['investment'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [14]:
X_train_fin, X_test_fin, y_train_fin, y_test_fin = train_test_split(df_fin['text_parsed'], 
                                                    df_fin['fintech'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [22]:
with open('Pickles/tfidf.pickle', 'rb') as data:
    tfidf = pickle.load(data)

def to_tfidf(X_train, y_train, X_test, y_test):
    features_train = tfidf.transform(X_train).toarray()
    labels_train = y_train
    print(features_train.shape)

    features_test = tfidf.transform(X_test).toarray()
    labels_test = y_test
    print(features_test.shape)
    
    return features_train, labels_train, features_test, labels_test

In [23]:
features_train_fin, labels_train_fin, features_test_fin, labels_test_fin = to_tfidf(X_train_fin, 
                                                                                   y_train_fin, 
                                                                                   X_test_fin, 
                                                                                   y_test_fin)
print('\n')

features_train_inv, labels_train_inv, features_test_inv, labels_test_inv = to_tfidf(X_train_inv, 
                                                                                   y_train_inv, 
                                                                                   X_test_inv, 
                                                                                   y_test_inv)

(223, 5000)
(40, 5000)


(238, 5000)
(42, 5000)


In [24]:
from sklearn.feature_selection import chi2
import numpy as np
def check_most_corr_words(features_train, labels_train, category_codes):
    for Product, category_id in sorted(category_codes.items()):
        features_chi2 = chi2(features_train, labels_train == category_id)
        indices = np.argsort(features_chi2[0])
        feature_names = np.array(tfidf.get_feature_names())[indices]
        unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
        print("# '{}' category:".format(Product))
        print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-10:])))
        print("")

In [25]:
check_most_corr_words(features_train_fin, labels_train_fin, {'fin': 1, 'not_fin': 0})

# 'fin' category:
  . Most correlated unigrams:
. паника
. венедиктов
. собственность
. венесуэла
. парад
. верификация
. заразить
. собор
. парламент
. грозный

# 'not_fin' category:
  . Most correlated unigrams:
. паника
. венедиктов
. собственность
. венесуэла
. парад
. верификация
. заразить
. собор
. парламент
. грозный



In [26]:
check_most_corr_words(features_train_inv, labels_train_inv, {'inv': 1, 'not_inv': 0})

# 'inv' category:
  . Most correlated unigrams:
. подъём
. подсайт
. подруга
. подозрительный
. австрия
. подмосковье
. подлодка
. подлинность
. пожар
. пародия

# 'not_inv' category:
  . Most correlated unigrams:
. подъём
. подсайт
. подруга
. подозрительный
. австрия
. подмосковье
. подлодка
. подлинность
. пожар
. пародия



In [27]:
# X_train
with open('Pickles/X_train_inv.pickle', 'wb') as output:
    pickle.dump(X_train_inv, output)
    
# X_test    
with open('Pickles/X_test_inv.pickle', 'wb') as output:
    pickle.dump(X_test_inv, output)
    
# y_train
with open('Pickles/y_train_inv.pickle', 'wb') as output:
    pickle.dump(y_train_inv, output)
    
# y_test
with open('Pickles/y_test_inv.pickle', 'wb') as output:
    pickle.dump(y_test_inv, output)
    
# features_train
with open('Pickles/features_train_inv.pickle', 'wb') as output:
    pickle.dump(features_train_inv, output)

# labels_train
with open('Pickles/labels_train_inv.pickle', 'wb') as output:
    pickle.dump(labels_train_inv, output)

# features_test
with open('Pickles/features_test_inv.pickle', 'wb') as output:
    pickle.dump(features_test_inv, output)

# labels_test
with open('Pickles/labels_test_inv.pickle', 'wb') as output:
    pickle.dump(labels_test_inv, output)

In [28]:
# X_train
with open('Pickles/X_train_fin.pickle', 'wb') as output:
    pickle.dump(X_train_fin, output)
    
# X_test    
with open('Pickles/X_test_fin.pickle', 'wb') as output:
    pickle.dump(X_test_fin, output)
    
# y_train
with open('Pickles/y_train_fin.pickle', 'wb') as output:
    pickle.dump(y_train_fin, output)
    
# y_test
with open('Pickles/y_test_fin.pickle', 'wb') as output:
    pickle.dump(y_test_fin, output)
    
# features_train
with open('Pickles/features_train_fin.pickle', 'wb') as output:
    pickle.dump(features_train_fin, output)

# labels_train
with open('Pickles/labels_train_fin.pickle', 'wb') as output:
    pickle.dump(labels_train_fin, output)

# features_test
with open('Pickles/features_test_fin.pickle', 'wb') as output:
    pickle.dump(features_test_fin, output)

# labels_test
with open('Pickles/labels_test_fin.pickle', 'wb') as output:
    pickle.dump(labels_test_fin, output)